In [ ]:
from copy import deepcopy
import logging
import os
import re
import json

import musicbrainzngs

In [ ]:
pattern = re.compile(
    r".*I used [@#]?Shazam to discover (?P<title>.*) by (?P<artist>.*)\. https.*"
)

In [ ]:
incs = 'artists, releases, discids, media, artist-credits, isrcs, work-level-rels, annotation, aliases, tags, user-tags, ratings, user-ratings, area-rels, artist-rels, label-rels, place-rels, event-rels, recording-rels, release-rels, release-group-rels, series-rels, url-rels, work-rels, instrument-rels'
incs = [i.strip() for i in incs.split(",")]

In [ ]:
def filter_recordings_by_artist(work, artist_id):
    for recording_info in work["recording-relation-list"]:
        recording = musicbrainzngs.get_recording_by_id(
            recording_info["recording"]["id"],
            includes=incs
        )

        for artist in recording["recording"]["artist-credit"]:
            try:
                if artist["artist"]["id"] == artist_id:
                    yield recording
                    break
            except TypeError:
                # WAT
                continue

In [ ]:
def get_earliest_year_from_recordings(recordings):
    years = set()
    for recording in recordings:
        for release in recording["recording"]["release-list"]:
            if "date" in release:
                years.add(int(release["date"][:4]))

    return min(years)

In [ ]:
def musicbrainz(string):
    musicbrainzngs.auth("blackhold", "qHu6JXtYzAmia4QvE2zndpjgg")
    musicbrainzngs.set_useragent("geosong", "2", "pycamp-es2022")

    song_title, artist_name = pattern.search(string).groups()

    related_artists = musicbrainzngs.search_artists(artist=artist_name, type="group")
    chosen_artist = max(related_artists["artist-list"], key=lambda a: int(a["ext:score"]))

    related_works = musicbrainzngs.search_works(song_title)
    work = max(related_works["work-list"], key=lambda w: int(w["ext:score"]))

    possible_recordings = filter_recordings_by_artist(work, chosen_artist["id"])
    year = get_earliest_year_from_recordings(possible_recordings)

    return {
        "artist": chosen_artist["name"],
        "title": work["title"],
        "year": year,
    }

In [ ]:
text = (
    'I used @Shazam to discover Yes Sir, I Can Boogie by Baccara. '
    'https://t.co/JaCq9MNPhM'
)

In [ ]:
musicbrainz(text)